In [2]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import pandas_gbq
import openai
from collections import deque



In [3]:
import nest_asyncio
nest_asyncio.apply()

In [1]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_dir='resort data', input_files=['resort data/Accommodation at Serenity Bay Resort.pdf']).load_data()


c:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
documents

[Document(id_='ecd1f141-d961-4c5b-8e49-0f26bdcd7bbe', embedding=None, metadata={'page_label': '1', 'file_name': 'Accommodation at Serenity Bay Resort.pdf', 'file_path': 'resort data\\Accommodation at Serenity Bay Resort.pdf', 'file_type': 'application/pdf', 'file_size': 61837, 'creation_date': '2024-06-08', 'last_modified_date': '2024-06-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Accommodation\nat\nSerenity\nBay\nResort\n1.\nRoom\nTypes\nand\nDescriptions\n1.1.\nGarden\nView\nRoom\n●\nDescription:\nCozy\nand\ncomfortable\nrooms\nwith\na\nbeautiful\nview\nof\nthe\nresort's\nlush\ngardens.\n●\nAmenities:\nKing-sized\nbed,\nprivate\nbalcony,\nair\nconditioning,\nfree\nWi-Fi,\nflat-screen\nTV,\nminibar,\nand\nen-suite\nbathroom\nwith\nrain\nshower

In [4]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [5]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


Settings.llm = OpenAI(model = "gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model = "text-embedding-ada-002")

In [6]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [9]:
summary_query_engine = summary_index.as_query_engine(response_mode = "tree_summarize", use_asynch = True)
vector_query_engine = vector_index.as_query_engine()

In [12]:
from llama_index.core.tools import QueryEngineTool
summary_tool = QueryEngineTool.from_defaults(
            query_engine = summary_query_engine, 
            description="Useful for retrieving summary of the documents"
            )
vector_tool = QueryEngineTool.from_defaults(
            query_engine = vector_query_engine, 
            description="Useful for retrieving specific context from the documents"
            )

In [13]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(selector=LLMSingleSelector.from_defaults(),
                  query_engine_tools=[summary_tool,vector_tool],
                  verbose=True
                  )

In [17]:
response = query_engine.query("What is the summary of the document")
print(response)

Selecting query engine 0: The summary of the document is a concise overview of the main points and information contained within the document, making it useful for retrieving the summary of the documents..
The document provides detailed information about the accommodation options available at Serenity Bay Resort, including room types, descriptions, amenities, sizes, rates, and packages. It also outlines the booking and reservation policies, cancellation and modification policies, check-in and check-out procedures, as well as additional information such as payment methods, special requests, and early check-in/late check-out options.


In [18]:
print(len(response.source_nodes))

3
